In [2]:
import pandas as pd
import numpy as np
import random
import re

In [2]:
## Read CSV file and remove column CD
df = pd.read_csv('ppp_2000.csv')
df = df.drop(columns=['CD'])

In [3]:
df

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender
0,57600.0,Honolulu,HI,96817,444130.0,Corporation,Asian,Male Owned,Non-Veteran,NaN,5.0,04/29/2020,Hawaii National Bank
1,18600.0,HONOLULU,HI,96817,621112.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,3.0,04/28/2020,"American Savings Bank, FSB"
2,123300.0,KANEOHE,HI,96744,238220.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,8.0,04/14/2020,Central Pacific Bank
3,68600.0,KIHEI,HI,96753,238220.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,5.0,04/05/2020,"American Savings Bank, FSB"
4,14995.0,Kailua Kona,HI,96740,713940.0,Self-Employed Individuals,Unanswered,Unanswered,Unanswered,NaN,1.0,05/14/2020,"Kabbage, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,38272.0,KAHULUI,HI,96732,621310.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,5.0,05/15/2020,Intuit Financing Inc.
1996,77900.0,KANEOHE,HI,96744,238210.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,6.0,04/09/2020,"American Savings Bank, FSB"
1997,78000.0,LAHAINA,HI,96761,487210.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,7.0,04/15/2020,First Hawaiian Bank
1998,32540.0,AIEA,HI,96701,621310.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,2.0,04/28/2020,Bank of Hawaii


In [4]:
## Inject non-ascii characters: [],..
def inject_regexr_error(df, column):
    rand_val = random.random()
    if rand_val<0.33:
        df[column] = df[column].apply(lambda x: f'[{x}]')
    elif rand_val<0.66:
        df[column]= df[column].apply(lambda x: f"{x};")
    else:
        pass
    return df

In [5]:
## Inject non-ascii chars to column LoanAmount
df = inject_regexr_error(df, 'LoanAmount')
df['LoanAmount']

0        57600.0
1        18600.0
2       123300.0
3        68600.0
4        14995.0
          ...   
1995     38272.0
1996     77900.0
1997     78000.0
1998     32540.0
1999     20832.0
Name: LoanAmount, Length: 2000, dtype: float64

In [6]:
## Inject extra whitespaces
def inject_white_spaces_error(df, column):
    rand_val = random.random()
    if rand_val<0.6:
        df[column]= df[column].apply(lambda x: f" {x}  ")
    return df

In [7]:
## inject values errors to column 'State'
def inject_state_error(value):
    rand_val = random.random()
    if rand_val<0.33:
        return ''
    elif rand_val<0.66:
        return 'Hawaii'
    else:
        return value

In [8]:
# inject missing values & Full names to column State
df['State'] = df['State'].apply(inject_state_error)

In [9]:
# randomly remove zipcode 
def remove_long_zip_values(df, column='Zip', threshold=10, drop_ratio=0.1, random_state=None):
    """
    Removes (blanks out) 10% of zip values with large group (group by zip the number>10).
    """
    if random_state is not None:
        random.seed(random_state)

    # Count how many times each zip appears
    zip_counts = df[column].value_counts()

    # Filter zips that occur more than min_group_size
    zips_to_modify = zip_counts[zip_counts > threshold].index

    # Copy to avoid modifying original DataFrame
    df = df.copy()

    for zip_val in zips_to_modify:
        indices = df[df[column] == zip_val].index.tolist()
        random.shuffle(indices)
        n_remove = int(len(indices) * drop_ratio)
        remove_indices = indices[:n_remove]
        df.loc[remove_indices, column] = ''

    return df

In [10]:
df = remove_long_zip_values(df, 'Zip')
df['Zip']

/var/folders/cf/91dwsy8s0dn1zlwwrzdf7zzc0000gn/T/ipykernel_89431/3471689746.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[remove_indices, column] = ''


0       96817
1       96817
2       96744
3            
4       96740
        ...  
1995    96732
1996    96744
1997    96761
1998    96701
1999    96755
Name: Zip, Length: 2000, dtype: object

In [11]:
def corrupt_honolulu_zip(df, zip_col='Zip', city_col='City', n_rows=10, random_state=None):
    """
    Corrupts zip codes for randomly selected 'Honolulu' rows (case-insensitive).
    Logs original and corrupted values.
    """
    if random_state is not None:
        random.seed(random_state)

    df = df.copy()
    log = []

    # 1. Find all Honolulu rows (case-insensitive)
    honolulu_mask = df[city_col].str.lower() == 'honolulu'
    honolulu_indices = df[honolulu_mask].index.tolist()

    # 2. Get non-Honolulu zips
    non_honolulu_zips = df[~honolulu_mask][zip_col].dropna().unique().tolist()
    if not non_honolulu_zips or not honolulu_indices:
        return df, [], pd.DataFrame(columns=["row_index", "original_zip", "corrupted_zip"])

    # 3. Sample up to n_rows from Honolulu rows
    selected_indices = random.sample(honolulu_indices, min(n_rows, len(honolulu_indices)))

    # 4. Replace selected zip values with random ones from non-Honolulu
    for idx in selected_indices:
        original_zip = df.at[idx, zip_col]
        corrupted_zip = random.choice(non_honolulu_zips)
        df.at[idx, zip_col] = corrupted_zip
        log.append({
            "row_index": idx,
            "original_zip": original_zip,
            "corrupted_zip": corrupted_zip
        })

    log_df = pd.DataFrame(log)
    return df, selected_indices, log_df

In [12]:
df, corrupted_rows, zip_log = corrupt_honolulu_zip(df, random_state=42)

# Save the log
zip_log.to_csv("honolulu_zip_replacement_log.csv", index=False)

# View result
print("Corrupted row indices:", corrupted_rows)
print(zip_log)

Corrupted row indices: [1505, 260, 71, 1720, 679, 606, 534, 325, 1709, 240]
   row_index original_zip  corrupted_zip
0       1505        96817          96815
1        260        96817          96720
2         71        96822          96826
3       1720        96819          96709
4        679        96818          96795
5        606        96813          96748
6        534                       96720
7        325        96822          96761
8       1709        96815          96706
9        240        96816          96727


In [13]:
# inject extra whitespaces to column city
df = inject_white_spaces_error(df, 'City')
df

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender
0,57600.0,Honolulu,HI,96817,444130.0,Corporation,Asian,Male Owned,Non-Veteran,NaN,5.0,04/29/2020,Hawaii National Bank
1,18600.0,HONOLULU,HI,96817,621112.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,3.0,04/28/2020,"American Savings Bank, FSB"
2,123300.0,KANEOHE,Hawaii,96744,238220.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,8.0,04/14/2020,Central Pacific Bank
3,68600.0,KIHEI,Hawaii,,238220.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,5.0,04/05/2020,"American Savings Bank, FSB"
4,14995.0,Kailua Kona,HI,96740,713940.0,Self-Employed Individuals,Unanswered,Unanswered,Unanswered,NaN,1.0,05/14/2020,"Kabbage, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,38272.0,KAHULUI,HI,96732,621310.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,5.0,05/15/2020,Intuit Financing Inc.
1996,77900.0,KANEOHE,,96744,238210.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,6.0,04/09/2020,"American Savings Bank, FSB"
1997,78000.0,LAHAINA,HI,96761,487210.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,7.0,04/15/2020,First Hawaiian Bank
1998,32540.0,AIEA,Hawaii,96701,621310.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,2.0,04/28/2020,Bank of Hawaii


In [14]:
### Inject acronym to column BusinessType

def inject_business_type_errors(df, column='BusinessType', acronym_ratio=0.1, case_ratio=0.4, random_state=None):
    """
    Injects errors into the BusinessType column by:
    - Replacing <acronym_ratio> of rows with acronyms
    - Randomly changing <case_ratio> of rows to uppercase or lowercase
    - Leaving the rest unchanged
    """
    if random_state is not None:
        random.seed(random_state)

    df = df.copy()
    valid_indices = df[df[column].notna()].index.tolist()
    random.shuffle(valid_indices)

    n_total = len(valid_indices)
    n_acronym = int(n_total * acronym_ratio)
    n_case = int(n_total * case_ratio)

    acronym_indices = valid_indices[:n_acronym]
    remaining_indices = [i for i in valid_indices if i not in acronym_indices]
    case_indices = random.sample(remaining_indices, n_case)

    # Predefined acronym map
    acronym_map = {
        "Corporation": "Corp",
        "Subchapter S Corporation": "S-Corp",
        "Self-Employed Individuals": "SEI",
        "Limited  Liability Company(LLC)": "LLC",
        "Sole Proprietorship": "SP",
        "Non-Profit Organization": "NPO",
        "Non-Profit Childcare Center": "NPCC",
        "Independent Contractors": "IC",
        "Limited Liability Partnership": "LLP",
        "Partnership": "P"
    }

    # Apply acronym changes
    for idx in acronym_indices:
        original = df.at[idx, column]
        df.at[idx, column] = acronym_map.get(original, original)

    # Apply case changes
    for idx in case_indices:
        val = df.at[idx, column]
        if isinstance(val, str):
            df.at[idx, column] = val.lower() if random.random() < 0.5 else val.upper()

    return df, {
        "acronymized_rows": n_acronym,
        "case_modified_rows": n_case,
        "untouched_rows": n_total - n_acronym - n_case
    }

In [15]:
df, summary = inject_business_type_errors(df, random_state=42)
summary

{'acronymized_rows': 200, 'case_modified_rows': 800, 'untouched_rows': 1000}

In [16]:
df

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender
0,57600.0,Honolulu,HI,96817,444130.0,CORPORATION,Asian,Male Owned,Non-Veteran,NaN,5.0,04/29/2020,Hawaii National Bank
1,18600.0,HONOLULU,HI,96817,621112.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,3.0,04/28/2020,"American Savings Bank, FSB"
2,123300.0,KANEOHE,Hawaii,96744,238220.0,CORPORATION,Unanswered,Unanswered,Unanswered,NaN,8.0,04/14/2020,Central Pacific Bank
3,68600.0,KIHEI,Hawaii,,238220.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,5.0,04/05/2020,"American Savings Bank, FSB"
4,14995.0,Kailua Kona,HI,96740,713940.0,self-employed individuals,Unanswered,Unanswered,Unanswered,NaN,1.0,05/14/2020,"Kabbage, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,38272.0,KAHULUI,HI,96732,621310.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,5.0,05/15/2020,Intuit Financing Inc.
1996,77900.0,KANEOHE,,96744,238210.0,limited liability company(llc),Unanswered,Unanswered,Unanswered,NaN,6.0,04/09/2020,"American Savings Bank, FSB"
1997,78000.0,LAHAINA,HI,96761,487210.0,corporation,Unanswered,Unanswered,Unanswered,NaN,7.0,04/15/2020,First Hawaiian Bank
1998,32540.0,AIEA,Hawaii,96701,621310.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,2.0,04/28/2020,Bank of Hawaii


In [17]:
df.to_csv("ppp_mar_26.csv", index=False)

In [18]:
### Start from ppp_mar_26.csv

In [3]:
df = pd.read_csv('ppp_mar_26.csv')

In [4]:
df

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender
0,57600.0,Honolulu,HI,96817.0,444130.0,CORPORATION,Asian,Male Owned,Non-Veteran,NaN,5.0,04/29/2020,Hawaii National Bank
1,18600.0,HONOLULU,HI,96817.0,621112.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,3.0,04/28/2020,"American Savings Bank, FSB"
2,123300.0,KANEOHE,Hawaii,96744.0,238220.0,CORPORATION,Unanswered,Unanswered,Unanswered,NaN,8.0,04/14/2020,Central Pacific Bank
3,68600.0,KIHEI,Hawaii,NaN,238220.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,5.0,04/05/2020,"American Savings Bank, FSB"
4,14995.0,Kailua Kona,HI,96740.0,713940.0,self-employed individuals,Unanswered,Unanswered,Unanswered,NaN,1.0,05/14/2020,"Kabbage, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,38272.0,KAHULUI,HI,96732.0,621310.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,5.0,05/15/2020,Intuit Financing Inc.
1996,77900.0,KANEOHE,NaN,96744.0,238210.0,limited liability company(llc),Unanswered,Unanswered,Unanswered,NaN,6.0,04/09/2020,"American Savings Bank, FSB"
1997,78000.0,LAHAINA,HI,96761.0,487210.0,corporation,Unanswered,Unanswered,Unanswered,NaN,7.0,04/15/2020,First Hawaiian Bank
1998,32540.0,AIEA,Hawaii,96701.0,621310.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,2.0,04/28/2020,Bank of Hawaii


In [8]:
## Inject informal date to DateApproved column. 
from datetime import datetime

def date_conversion(df, date_col):
    """
    Randomly converts about half of the dates to '%Y-%m-%d'. 
    The rest remain as '%m/%d/%Y'.

    Parameters:
        df (pd.DataFrame): Input DataFrame.
        date_col (str): Column with dates in '%m/%d/%Y' format.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Modified DataFrame and change log.
    """
    log = []
    modified_dates = []

    for idx, val in df[date_col].items():
        if random.random() < 0.5:
            try:
                dt = datetime.strptime(val, "%m/%d/%Y")
                new_val = dt.strftime("%Y-%m-%d")
                log.append({'Index': idx, 'Original': val, 'Modified': new_val})
                modified_dates.append(new_val)
            except ValueError:
                modified_dates.append(val)  # Keep original if parsing fails
        else:
            modified_dates.append(val)

    df[date_col] = modified_dates
    log_df = pd.DataFrame(log)
    return df, log_df

In [9]:
df, logs = date_conversion(df, 'DateApproved')

In [10]:
df

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender
0,57600.0,Honolulu,HI,96817.0,444130.0,CORPORATION,Asian,Male Owned,Non-Veteran,NaN,5.0,04/29/2020,Hawaii National Bank
1,18600.0,HONOLULU,HI,96817.0,621112.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,3.0,2020-04-28,"American Savings Bank, FSB"
2,123300.0,KANEOHE,Hawaii,96744.0,238220.0,CORPORATION,Unanswered,Unanswered,Unanswered,NaN,8.0,2020-04-14,Central Pacific Bank
3,68600.0,KIHEI,Hawaii,NaN,238220.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,5.0,04/05/2020,"American Savings Bank, FSB"
4,14995.0,Kailua Kona,HI,96740.0,713940.0,self-employed individuals,Unanswered,Unanswered,Unanswered,NaN,1.0,2020-05-14,"Kabbage, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,38272.0,KAHULUI,HI,96732.0,621310.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,5.0,2020-05-15,Intuit Financing Inc.
1996,77900.0,KANEOHE,NaN,96744.0,238210.0,limited liability company(llc),Unanswered,Unanswered,Unanswered,NaN,6.0,2020-04-09,"American Savings Bank, FSB"
1997,78000.0,LAHAINA,HI,96761.0,487210.0,corporation,Unanswered,Unanswered,Unanswered,NaN,7.0,2020-04-15,First Hawaiian Bank
1998,32540.0,AIEA,Hawaii,96701.0,621310.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,2.0,04/28/2020,Bank of Hawaii


In [13]:
logs.to_csv('date_convert.csv', index=False)

In [14]:
df.to_csv('ppp_errors.csv')